In [4]:
import os
import epr
import sys

In [9]:
from pylab import *
from scipy.interpolate import RectBivariateSpline

<function numpy.core.numeric.asarray>

In [13]:
from scipy.interpolate import RectBivariateSpline

In [32]:
!ls -la /media/SOLabNFS2/store/model/ncep/gfs/gfs20110105/gfs.t06z.master.grbf00
# /model/ncep/gfs/gfs20110105/gfs.t12z.master.grbf00

-rwxr-xr-x 1 1111 1111 3250361 Jan  5  2011 /media/SOLabNFS2/store/model/ncep/gfs/gfs20110105/gfs.t06z.master.grbf00


In [14]:
def imresize(image, size):
    """
    Resizes coefficient arrays using bivariate spline approximation.
    """
    m, n = image.shape
    X = linspace(0, m - 1, size[0])
    Y = linspace(0, n - 1, size[1])
    kx, ky = min([m - 1, 3]), min([n - 1, 3])
    interp = RectBivariateSpline(
        arange(m), arange(n), image, kx=kx, ky=ky)
    resized = interp(X, Y)
    return resized

In [29]:
def ncepGFSmodel(startTime, lats_2, lons_2):
    """NCEP GFS model wind for givven time, lat/lon crop

    """
    ncepGFSmodel = {}  # empty dict for ncepGFSmodel

    iPath_wind = '/media/SOLabNFS2/store/model/ncep/gfs/'

    # find the ncep gfs filename to open from ASAR filename
    baseHour = floor((startTime.hour+3/2)/6)*6
    baseHour = min(18, baseHour)
    if startTime.hour-baseHour > 1.5:
        forecastHour = 3
    else:
        forecastHour = 0

    if startTime <= datetime.datetime(2014, 8, 19):
        ncepFileName = 'gfs' + startTime.strftime("%Y%m%d") + '/gfs.t' +\
                       '%.2d' % (baseHour) + 'z.master.grbf' +\
                       '%.2d' % (forecastHour)
        ncepFileName = 'gfs20110105/gfs.t06z.master.grbf00'

        try:
            grbs = pygrib.open(iPath_wind + ncepFileName)
        except:
            print "File not found %s" % (iPath_wind + ncepFileName)
            return False

        u_wind = None
        v_wind = None

        # wind contains u=u_wind.values[:], Lats=u_wind.latlons()[0], Lons=u_wind.latlons()[1]
        for idx, msg_info in enumerate(grbs.select()):
            if msg_info['short_name'] == '10u':
                u_wind = grbs.message(idx + 1)
            elif msg_info['short_name'] == '10v':
                v_wind = grbs.message(idx + 1)

        u = u_wind.values[:]
        v = v_wind.values[:]
        lats_wind = u_wind.latlons()[0]
        lons_wind = u_wind.latlons()[1]
    else:
        ncepFileName = 'gfs.' + startTime.strftime("%Y%m%d") +\
                       '%.2d' % (baseHour) + '/gfs.t' + '%.2d' % (baseHour) +\
                       'z.master.grbf' + '%.2d' % (forecastHour) +\
                       '.10m.uv.grib2'

        grbs = pygrib.open(iPath_wind + ncepFileName)

        u_wind = grbs.message(1)
        v_wind = grbs.message(2)
        u = u_wind['values']
        v = v_wind['values']
        lats_wind = u_wind['latitudes']
        lons_wind = u_wind['longitudes']
        lons_wind = reshape(lons_wind, (lons_wind.shape[0]/720, 720))
        lats_wind = reshape(lats_wind, (lats_wind.shape[0]/720, 720))

    # Make sure the longitude is between -180.00 .. 179.9
    lons_wind = map(
        lambda x:
        (lons_wind.ravel()[x]+180)-int((lons_wind.ravel()[x]+180)/360)*360-180,
        range(0, lons_wind.size)
    )
    lons_wind = reshape(lons_wind, lats_wind.shape)
    # plt.close('all')
    # plt.imshow(lons_wind)
    # plt.colorbar()

#     #Make sure the latitudes is between -90.00 .. 89.9, starting from North - positive
#     lats_wind = map(lambda x : (lats_wind.ravel()[x]+90)-int((lats_wind.ravel()[x]+90)/180)*180-90, xrange(0,lats_wind.size))
#     lats_wind = reshape(lats_wind, lons_wind.shape)
#     if lats_wind[0,0] < lats_wind[-1,-1]:
#         lats_wind = flipud(lats_wind)
#         u = flipud(u)
#         v = flipud(v)
#     plt.close('all')
#     plt.imshow(lats_wind)
#     plt.colorbar()

    # find subset
    res = findSubsetIndices(lats_2.min(), lats_2.max(), lons_2.min(),
                            lons_2.max(), lats_wind[:, 0], lons_wind[0, :])
    # expand subset by 1 pixel for better further pyresample
    res[0] = res[0]-2
    res[1] = res[1]+2
    res[2] = res[2]-2
    res[3] = res[3]+2

    # crop the data
    u = u[int(res[2]):int(res[3]), int(res[0]):int(res[1])]
    v = v[int(res[2]):int(res[3]), int(res[0]):int(res[1])]
    ncepGFSmodel['lats_wind'] = lats_wind[int(res[2]):int(res[3]),
                                          int(res[0]):int(res[1])]
    ncepGFSmodel['lons_wind'] = lons_wind[int(res[2]):int(res[3]),
                                          int(res[0]):int(res[1])]

    ncepGFSmodel['wind_dir'] = windDirection(u, v)
    ncepGFSmodel['wind_speed'] = sqrt(u**2 + v**2)

#     del u_wind, v_wind
    return ncepGFSmodel

In [62]:
iPath = '/media/SOLabNFS2/store/satellite/asar/2011/005/'
fileName = 'ASA_IMM_1PNPDK20110105_112327_000001123098_00138_46273_4557.N1'
# ASA_IMM_1PNPDK20110105_112327_000001123098_00138_46273_4557.N1

print os.path.join(iPath, fileName)
try:
    product = epr.Product(os.path.join(iPath, fileName))
except:
    print 'unable to read file'
    sys.exit()

try:
    band = product.get_band('proc_data')
except epr.EPRValueError:
    print 'unable to get band "proc_data": epr_get_band_id: band not found'
    sys.exit()

sc_w = product.get_scene_width()
sc_h = product.get_scene_height()

print 'sc_w*sc_h = ', sc_w * sc_h
# if sc_w*sc_h > 30000000:
if sc_w*sc_h > 24000000:
    print "ASAR Image too large, skipping"
    sys.exit()

try:
    raw_counts = band.read_as_array(sc_w, sc_h)  # , xstep=4, ystep=4)

    incident_angle = product.get_band('incident_angle').read_as_array(sc_w,
                                                                      sc_h)
    # , xstep=4, ystep=4)
except epr.EPRValueError:
    print "EPRValueError"
    sys.exit()

# Get lat/lon from geolocation grid
dataset = product.get_dataset('GEOLOCATION_GRID_ADS')
fltp_lats = map(
    lambda x:
    dataset.read_record(x).get_field('first_line_tie_points.lats').get_elems(),
    range(dataset.get_num_records())
)
lltp_lats = map(
    lambda x:
    dataset.read_record(x).get_field('last_line_tie_points.lats').get_elems(),
    range(dataset.get_num_records())
)
fltp_lons = map(
    lambda x:
    dataset.read_record(x).get_field('first_line_tie_points.longs').get_elems(),
    range(dataset.get_num_records())
)
lltp_lons = map(
    lambda x:
    dataset.read_record(x).get_field('last_line_tie_points.longs').get_elems(),
    range(dataset.get_num_records())
)

fltp_lats = asarray(double(fltp_lats))/1e6
lltp_lats = asarray(double(lltp_lats))/1e6
fltp_lons = asarray(double(fltp_lons))/1e6
lltp_lons = asarray(double(lltp_lons))/1e6

lats = row_stack((fltp_lats, lltp_lats[-1, :]))
lons = row_stack((fltp_lons, lltp_lons[-1, :]))

lats = fliplr(lats)
lons = fliplr(lons)

lats_2 = imresize(lats, raw_counts.shape)
lons_2 = imresize(lons, raw_counts.shape)

# if lats.max() <= 35:
#     print "skipping no area overlap"
#     return False

# for p in oPaths:
#     for sp in p:
#         mkdirs(sp)

# Trimming the array by removing zero values from rows and cols
msk = []
for m in range(raw_counts.shape[0]):
    if raw_counts[m, :].sum() == 0:
        msk.append(m)
raw_counts = delete(raw_counts, msk, axis=0)
lats_2 = delete(lats_2, msk, axis=0)
lons_2 = delete(lons_2, msk, axis=0)
incident_angle = delete(incident_angle, msk, axis=0)

msk = []
for n in range(raw_counts.shape[1]):
    if raw_counts[:, n].sum() == 0:
        msk.append(n)
raw_counts = delete(raw_counts, msk, axis=1)
lats_2 = delete(lats_2, msk, axis=1)
lons_2 = delete(lons_2, msk, axis=1)
incident_angle = delete(incident_angle, msk, axis=1)

# # Adding Sigma_0
# calibration_constant = \
# product.get_dataset('MAIN_PROCESSING_PARAMS_ADS').read_record(0).get_field('calibration_factors.1.ext_cal_fact').get_elems()
# sigma0 = 10*log10( raw_counts**2*sin(incident_angle*pi/180)/calibration_constant )
# sigma0 = raw_counts**2*sin(incident_angle*pi/180)/calibration_constant
# 
# from scipy.signal import wiener
# sigma0w = wiener(sigma0, mysize=(7, 7), noise=None)
# # sigma0w = sigma0

# # Nice Image (roughness)
# pol = product.get_sph().get_field('MDS1_TX_RX_POLAR').get_elem()
# if pol == 'H/H':
#     ph = (2.20495, -14.3561e-2, 11.28e-4)
#     sigma0_hh_ref = exp((ph[0]+incident_angle*ph[1] +
#                          incident_angle**2*ph[2])*log(10))
#     roughness = sigma0w/sigma0_hh_ref
#     del sigma0_hh_ref
# elif pol == 'V/V':
#     pv = (2.29373, -15.393e-2, 15.1762e-4)
#     sigma0_vv_ref = exp((pv[0]+incident_angle*pv[1] +
#                          incident_angle**2*pv[2])*log(10))
#     roughness = sigma0w/sigma0_vv_ref
#     del sigma0_vv_ref

# Adding Model wind
# startTime = datetime.datetime.strptime(fileName[14:29], "%Y%m%d_%H%M%S")
# ncepGFSmodelWind = ncepGFSmodel(startTime, lats_2, lons_2)
# if not ncepGFSmodelWind:
#     sys.exit()

# Reprojecting data

# Pixel resolution
# we use pxlResWind/pxlResSAR for further pyresample radius_of_influence and sigmas
# pxlResWind = asarray(
#     distancelib.getPixelResolution(ncepGFSmodelWind['lats_wind'],
#                                    ncepGFSmodelWind['lons_wind'],
#                                    ncepGFSmodelWind['lats_wind'].shape,
#                                    'km')
# )
pxlResSAR = asarray(
    distancelib.getPixelResolution(lats_2, lons_2, lons_2.shape, 'km')
)*1e3


/media/SOLabNFS2/store/satellite/asar/2011/005/ASA_IMM_1PNPDK20110105_112327_000001123098_00138_46273_4557.N1
sc_w*sc_h =  10068300


In [59]:
# sys.path.append('/media/SOLabNFS2/tmp/posada/handlers/')
# print sys.path
# reload(cmod)

['', '/usr/local/lib/python2.7/dist-packages/matplotlib-1.5.dev1-py2.7-linux-x86_64.egg', '/usr/local/lib/python2.7/dist-packages/pykdtree-1.0-py2.7-linux-x86_64.egg', '/usr/local/lib/python2.7/dist-packages/pyepr-0.9.4.dev0-py2.7-linux-x86_64.egg', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-linux2', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/PIL', '/usr/lib/python2.7/dist-packages/gst-0.10', '/usr/lib/python2.7/dist-packages/gtk-2.0', '/usr/lib/pymodules/python2.7', '/usr/lib/python2.7/dist-packages/ubuntu-sso-client', '/usr/local/lib/python2.7/dist-packages/IPython/extensions', '/media/SOLabNFS2/tmp/posada/handlers/']


<module 'cmod' from '/media/SOLabNFS2/tmp/posada/handlers/cmod/__init__.py'>

In [61]:
# sys.path.append('/media/SOLabNFS2/tmp/posada/handlers/')
# from cmod import distancelib

In [66]:
ncep_def = pr.geometry.GridDefinition(lons=ncepGFSmodelWind['lons_wind'],
                                      lats=ncepGFSmodelWind['lats_wind'])
swath_def = pr.geometry.SwathDefinition(lons=lons_2, lats=lats_2)

# wind_speed_model_swath = pr.kd_tree.resample_gauss(
#     ncep_def, ncepGFSmodelWind['wind_speed'].ravel(), swath_def,
#     radius_of_influence=2*pxlResWind.max()*1e3, neighbours=12,
#     sigmas=pxlResWind.max()*1e3, fill_value=None, nprocs=numProcs
# )
wind_dir_model_swath = pr.kd_tree.resample_gauss(
    ncep_def, ncepGFSmodelWind['wind_dir'].ravel(), swath_def,
    radius_of_influence=2*pxlResWind.max()*1e3, neighbours=12,
    sigmas=pxlResWind.max()*1e3, fill_value=None, nprocs=numProcs
)

# calculate bearing from initial lats/lons for further wind calculation
bearing = zeros((lons.shape[0]-1, lons.shape[1]))

for n in range(0, lons.shape[1]):
    col = ([lats[:-1, n], lons[:-1, n]], [lats[1:, n], lons[1:, n]])
    for m in range(0, lons.shape[0]-1):
        bearing[m][n] = distancelib.bearing(asarray(col[0])[:, m],
                                            asarray(col[1])[:, m])

# interpolate to raw_counts.shape
bearing_2 = imresize(bearing, raw_counts.shape)

# NB! WINDDIR = 0 WHEN WIND BLOWS TOWARDS RADAR!
wind_dir_model_swath_rel = 90 + bearing_2 - wind_dir_model_swath

if pol == 'H/H':
    PR = PR_Mouche(incident_angle, wind_dir_model_swath_rel)
    try:
        from cmod_gpu import rcs2windOpenCl
        wind_speed_asar = rcs2windOpenCl(sar=sigma0w*PR,
                                         windir=wind_dir_model_swath_rel,
                                         theta=incident_angle)
    except Exception:
        from cmod_vect import rcs2windPar
        wind_speed_asar = rcs2windPar(sigma0w*PR, cmdv=5,
                                      windir=wind_dir_model_swath_rel,
                                      theta=incident_angle,
                                      nprocs=numProcs)
elif pol == 'V/V':
    try:
        from cmod_gpu import rcs2windOpenCl
        wind_speed_asar = rcs2windOpenCl(sar=sigma0w,
                                         windir=wind_dir_model_swath_rel,
                                         theta=incident_angle)
    except Exception:
        from cmod_vect import rcs2windPar
        wind_speed_asar = rcs2windPar(sigma0w, cmdv=5,
                                      windir=wind_dir_model_swath_rel,
                                      theta=incident_angle,
                                      nprocs=numProcs)

del lats, lons
del sigma0, raw_counts
del bearing
del ncepGFSmodelWind
gc.collect()

for proj in ['EPSG:4326', 'EPSG:3413']:
    # for proj in ['EPSG:4326']:
    print "    start projection %s" % proj
    if proj == 'EPSG:4326':
        oPath = oPath_4326
        for outpath in oPath:
            mkdirs(outpath)
        area_def = swath_area_def(
            name='Temporal SWATH EPSG Projection 4326', proj='eqc',
            lonlim=(lons_2.min(), lons_2.max()),
            latlim=(lats_2.min(), lats_2.max()),
            ellps="WGS84", res=pxlResSAR.max()
        )
        # Set the parameters for GSHHS masking
        proj_ = '4326'
        proj_name = None
        units = 'deg'
    elif proj == 'EPSG:3413':
        oPath = oPath_3413
        for outpath in oPath:
            mkdirs(outpath)
        area_def = swath_area_def(
            name='Temporal SWATH EPSG Projection 3413', proj='stere',
            lonlim=(lons_2.min(), lons_2.max()),
            latlim=(lats_2.min(), lats_2.max()),
            ellps="WGS84", res=pxlResSAR.max(),
            lat_ts=70, lat_0=90, lon_0=-45
        )
        # Set the parameters for GSHHS masking
        proj_ = '+units=m +ellps=WGS84 +lon_0=-45 +proj=stere +lat_ts=70 +lat_0=90'
        proj_name = '3413'
        units = 'm'

    print area_def
    print "roughness.shape = ", roughness.shape

    roughness_res = pr.kd_tree.resample_nearest(
        swath_def, roughness.ravel(), area_def,
        radius_of_influence=pxlResSAR.max(),
        epsilon=0.5, nprocs=numProcs, fill_value=None
    )
    print "resample_nearest done"